## Geocoding and directions for tourist attractions in Austria

##### The workflow order is as follows:
1. Import libraries
2. Import addresses
3. Geocode addresses
4. Plot addresses
5. Select start and end point to plot route

In [1]:
import openrouteservice as ors
from openrouteservice.directions import directions
import folium
import geopy
import leafmap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io

In [ ]:
pip install openrouteservice

  Using cached openrouteservice-2.3.3-py3-none-any.whl (33 kB)
Note: you may need to restart the kernel to use updated packages.


##### API Key

In [2]:
import config
# performs requests to the ORS API services
# client will be used in all examples
client = ors.Client(key=config.api_key)

#### Import addresses

Import addresses in CSV file format 

In [3]:
upload = widgets.FileUpload(
        accept='*.csv',
        multiple=False)
display(upload)

FileUpload(value={}, accept='*.csv', description='Upload')

#### Data Cleaning 
Inspect the imported data and optimize it for geocoding

In [6]:
input_file = list(upload.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)
df.head()

,Unnamed: 0,restaurant,street,zip,city,country
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany


In [7]:
# Merge street, zip and city fields into an address field Using DataFrame.apply() and lambda function

df["address"] = df["street"].map(str) + ', ' + df["zip"].map(str)+ ', ' + df["city"]+ ', ' +  df['country']
df.head()

,Unnamed: 0,restaurant,street,zip,city,country,address
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"S-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger..."
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"U-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger..."
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany"
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany"
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany"


#### Geocode addresses
Geocode addresses using geopy library and Nominatim

In [8]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [9]:
geolocator = Nominatim(user_agent="Practice")

In [10]:
locator = Nominatim(user_agent="Practice")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
df.head()

,Unnamed: 0,restaurant,street,zip,city,country,address,location,point
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"S-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger...",None,None
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"U-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger...",None,None
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany","(Europaplatz 1, 10557 Berlin, Europaplatz, Moa...","(52.5257446, 13.3685935, 0.0)"
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany","(Schloss-Straßen-Center, 1, Walther-Schreiber-...","(52.46544755, 13.327684176306294, 0.0)"
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany","(1, Hansastraße, Alt-Hohenschönhausen, Lichten...","(52.5470223, 13.4684079, 0.0)"


Drop any colummns with null coordinate values

In [16]:
df = df.dropna()
df1.head()

,Unnamed: 0,restaurant,street,zip,city,country,address,location,point
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany","(Europaplatz 1, 10557 Berlin, Europaplatz, Moa...","(52.5257446, 13.3685935, 0.0)"
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany","(Schloss-Straßen-Center, 1, Walther-Schreiber-...","(52.46544755, 13.327684176306294, 0.0)"
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany","(1, Hansastraße, Alt-Hohenschönhausen, Lichten...","(52.5470223, 13.4684079, 0.0)"
5,5,McDonalds,Hanne-Sobek-Platz 1,13357,Berlin,Germany,"Hanne-Sobek-Platz 1, 13357, Berlin, Germany","(Hanne-Sobek-Platz, Gesundbrunnen, Mitte, Berl...","(52.5490394, 13.3874026, 0.0)"
6,6,McDonalds,Grunerstr. 20,10179,Berlin,Germany,"Grunerstr. 20, 10179, Berlin, Germany","(Alexa, 20, Grunerstraße, Mitte, Berlin, 10179...","(52.51907225, 13.416015406346249, 0.0)"


In [17]:
df.shape

(136, 9)

In [18]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

,Unnamed: 0,restaurant,street,zip,city,country,address,location,point,latitude,longitude,altitude
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany","(Europaplatz 1, 10557 Berlin, Europaplatz, Moa...","(52.5257446, 13.3685935, 0.0)",52.525745,13.368593,0.0
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany","(Schloss-Straßen-Center, 1, Walther-Schreiber-...","(52.46544755, 13.327684176306294, 0.0)",52.465448,13.327684,0.0
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany","(1, Hansastraße, Alt-Hohenschönhausen, Lichten...","(52.5470223, 13.4684079, 0.0)",52.547022,13.468408,0.0
5,5,McDonalds,Hanne-Sobek-Platz 1,13357,Berlin,Germany,"Hanne-Sobek-Platz 1, 13357, Berlin, Germany","(Hanne-Sobek-Platz, Gesundbrunnen, Mitte, Berl...","(52.5490394, 13.3874026, 0.0)",52.549039,13.387403,0.0
6,6,McDonalds,Grunerstr. 20,10179,Berlin,Germany,"Grunerstr. 20, 10179, Berlin, Germany","(Alexa, 20, Grunerstraße, Mitte, Berlin, 10179...","(52.51907225, 13.416015406346249, 0.0)",52.519072,13.416015,0.0


In [20]:
df = df.drop(['street', 'zip', 'city', 'point'], axis=1)

In [21]:
df.head()

,Unnamed: 0,restaurant,country,address,location,latitude,longitude,altitude
2,2,McDonalds,Germany,"Europaplatz 1, 10557, Berlin, Germany","(Europaplatz 1, 10557 Berlin, Europaplatz, Moa...",52.525745,13.368593,0.0
3,3,McDonalds,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany","(Schloss-Straßen-Center, 1, Walther-Schreiber-...",52.465448,13.327684,0.0
4,4,McDonalds,Germany,"Hansastr. 1, 13053, Berlin, Germany","(1, Hansastraße, Alt-Hohenschönhausen, Lichten...",52.547022,13.468408,0.0
5,5,McDonalds,Germany,"Hanne-Sobek-Platz 1, 13357, Berlin, Germany","(Hanne-Sobek-Platz, Gesundbrunnen, Mitte, Berl...",52.549039,13.387403,0.0
6,6,McDonalds,Germany,"Grunerstr. 20, 10179, Berlin, Germany","(Alexa, 20, Grunerstraße, Mitte, Berlin, 10179...",52.519072,13.416015,0.0


#### Plot addresses
Plot geocoded addresses using folium and leafmap mapping libraries

In [52]:
map1 = folium.Map(
    location=[50.1109,8.6821],
    tiles='openstreetmap',
    zoom_start=6,
)

df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [57]:
Map = leafmap.Map(center=(50.1109,8.6821), zoom=5)
Map.add_xy_data(df, x="longitude", y="latitude", layer_name="restaurants")
Map

Map(center=[50.1109, 8.6821], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

#### Routing from start point to end point
Implement routing using openroutes service library

# widgets
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

# function
# calculate distance from a to b
# display map with ant path from a to b
def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    
    m = Map(center=(50.1109,8.6821), zoom=5, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles

# interaction between function and widget
widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

In [68]:
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
       
# interaction between function and widget
widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[50.1109,8.6821], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions

interactive(children=(Dropdown(description='start:', options=('Europaplatz 1, 10557, Berlin, Germany', 'Walthe…

In [ ]:
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    coordinates = [[start_lat, start_long], [end_lat, end_long]]

    route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')


    map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)
    folium.GeoJson(route, name='route').add_to(map_directions)

    folium.LayerControl().add_to(map_directions)

    map_directions
    
    
    m = Map(center=(38, -98), zoom=4, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles




# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions





# interaction between function and widget
ipywidgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

# coordinates from Nashville, TN (-86.781247, 36.163532) to Miami, FL (-80.191850, 25.771645)
# order for coordinates is [lon, lat]
coordinates = [[-86.781247, 36.163532], [-80.191850, 25.771645]]

# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions